<a href="https://colab.research.google.com/github/288756/VisArtificial/blob/master/P5_Segmentaci%C3%B3n_sem%C3%A1ntica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Práctica 5: Segmentación semántica**

La **segmentación semántica** consiste identificar la ubicación de un objeto en una imagen y marcarlo con una caja delimitadora o *bounding box*. Es habitual que, además de localizar el objeto, se clasifique en una de las clases objetivo (clasificación y localización). Por su parte, el proceso de localización implica determinar las coordenadas del *bounding box*, por lo que se puede abordar como un problema de regresión.

A continuación, vamos a ver un ejemplo de segmentación semántica utilizando la librería [TensorFlow](https://www.tensorflow.org/). Para ello utilizaremos una red pre-entrenada en ImageNet como extractor de características y resolveremos un problema de clasificación a nivel de píxel.

Antes de empezar, vamos a utilizar el método [set_random_seed()](https://www.tensorflow.org/api_docs/python/tf/keras/utils/set_random_seed) para establecer el valor de la **semilla** y garantizar la reproducibilidad de los resultados.

In [ ]:
from tensorflow.keras.utils import set_random_seed

seed = 121
set_random_seed(seed)  # establece todas las semillas aleatorias del programa (Python, NumPy y TensorFlow)

### **1. Conjunto de datos**

En esta práctica vamos a utilizar un conjunto para segmentación semántica denominado **UNIMIB**, adaptado de la versión original [UNIMIB 2016 Food Database](https://zenodo.org/records/4126613). Para ello, es necesario subir a Google Drive el fichero [UNIMIB2016_SS.zip](https://drive.google.com/file/d/1Rf34GIIcfSVM1_6pHQuxmUz_Vgy_0tCv/view?usp=sharing) que contiene:

*   Una carpeta `images` con dos directorios que contienen las imágenes originales:
> *  `train`, con 650 imágenes de entrenamiento.
> *  `val`, con 360 imágenes de validación.

*   Una carpeta `masks` con los mismos directorios, uno para cada partición de los datos, en los que hay una máscara de segmentación para cada imagen original.

El conjunto incluye regiones de 66 clases diferentes, 65 de las cuales corresponden a platos de comida y 1 al fondo de la imagen.


In [ ]:
from google.colab import drive

# Montar el Google Drive en el directorio del proyecto y descomprimir el fichero con los datos
drive.mount('/content/gdrive')
!unzip -n '/content/gdrive/My Drive/datasets/UNIMIB2016_SS.zip' >> /dev/null  # ACTUALIZAR: ruta al fichero comprimido

Una vez hemos descomprimido el fichero, vamos a especificar la información dependiente del conjunto de datos y a crear cuatro variables que contengan la ruta a los ficheros con las imágenes originales y las máscaras, para las particiones de entrenamiento y validación.

In [ ]:
import os
from glob import glob

# Especificar información dependiente del conjunto de datos
images_path = 'UNIMIB2016_SS/images/'
masks_path = 'UNIMIB2016_SS/masks/'
training_path = "train/"
validation_path = "val/"

img_width = img_height = 224  # dimensiones de la imagen
n_channels = 3                # número de canales
n_classes =66                 # número de clases

x_train = sorted(glob(os.path.join(images_path + training_path, "*.png")))
y_train = sorted(glob(os.path.join(masks_path + training_path, "*.png")))
n_samples_train = len(x_train)

x_val = sorted(glob(os.path.join(images_path + validation_path, "*.png")))
y_val = sorted(glob(os.path.join(masks_path + validation_path, "*.png")))
n_samples_val = len(x_val)

print("Número de ejemplos del conjunto de entrenamiento: ", n_samples_train)
print("Número de ejemplos del conjunto de validación: ", n_samples_val)

#### **1.1. Procesamiento de imágenes**

El siguiente paso consiste en procesar las imágenes y las máscaras. Para ello definiremos, en primer lugar, dos funciones que leen y pre-procesan, respectivamente, las imágenes y las máscaras:

*  Redimensionar y normalizar las imágenes.
*  Redimensionar las máscaras con un tipo de interpolación adecuado.

In [ ]:
import cv2

def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (img_width, img_height))
    x = x/255.0  # normalización
    return x

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (img_width, img_height), interpolation = cv2.INTER_NEAREST)  # interpolación sin solapamiento
    x = np.expand_dims(x, axis=-1)
    return x

A continuación, vamos a definir dos funciones para generar los conjuntos de entrenamiento y validación utilizando la clase [`Dataset`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset).

In [ ]:
import tensorflow as tf

def tf_parse(x, y):

    # Función interna de apoyo
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float64, tf.uint8])
    x.set_shape([img_width, img_height, 3])  # imágenes RGB
    y.set_shape([img_width, img_height, 1])  # máscaras (id labels)
    return x, y

def tf_dataset_training(x, y, batch):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.shuffle(n_samples_train)
    dataset = dataset.repeat()
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(1)
    return dataset

def tf_dataset_validation(x, y, batch):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.repeat()
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(1)
    return dataset

Por último, crearemos los conjuntos de datos y prepararemos los lotes haciendo uso de las funciones anteriores.

In [ ]:
# Crear los conjuntos de datos y preparar los lotes
batch_size = 2
train_dataset = tf_dataset_training(x_train, y_train, batch_size)
val_dataset = tf_dataset_validation(x_val, y_val, batch_size)

#### **1.2. Visualización de imágenes**

En esta sección visualizaremos algunas imágenes del conjunto de entrenamiento junto con sus máscaras de segmentación.

Para ello definiremos, en primer lugar, una función que nos permita leer las imágenes para mostrarlas correctamente con la librería [Matplotlib](https://matplotlib.org/).

In [ ]:
def read_and_rgb(x):
  x = cv2.imread(x)
  x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
  return x

A continuación, representaremos las cuatro primeras imágenes del conjunto de entrenamiento con sus respectivas máscaras de segmentación, utilizando un formato 8x8.

In [ ]:
import matplotlib.pyplot as plt

# Imágenes originales
fig = plt.figure(figsize=(15, 15))
a = fig.add_subplot(1, 4, 1)
imgplot = plt.imshow(read_and_rgb(x_train[0]))

a = fig.add_subplot(1, 4, 2)
imgplot = plt.imshow(read_and_rgb(x_train[1]))
imgplot.set_clim(0.0, 0.7)

a = fig.add_subplot(1, 4, 3)
imgplot = plt.imshow(read_and_rgb(x_train[2]))
imgplot.set_clim(0.0, 1.4)

a = fig.add_subplot(1, 4, 4)
imgplot = plt.imshow(read_and_rgb(x_train[3]))
imgplot.set_clim(0.0, 2.1)

# Máscaras de segmentación
fig = plt.figure(figsize=(15, 15))
a = fig.add_subplot(1, 4, 1)
imgplot = plt.imshow(read_and_rgb(y_train[0]))

a = fig.add_subplot(1, 4, 2)
imgplot = plt.imshow(read_and_rgb(y_train[1]))
imgplot.set_clim(0.0, 0.7)

a = fig.add_subplot(1, 4, 3)
imgplot = plt.imshow(read_and_rgb(y_train[2]))
imgplot.set_clim(0.0, 1.4)

a = fig.add_subplot(1, 4, 4)
imgplot = plt.imshow(read_and_rgb(y_train[3]))
imgplot.set_clim(0.0, 1.4)

### **2. Red convolucional**

La CNN que vamos a definir está compuesta por la base convolucional de una [MobileNetV2](https://www.tensorflow.org/api_docs/python/tf/keras/applications/mobilenet_v2), pre-entrenada con [ImageNet](https://www.image-net.org/). Esta parte del modelo constituye el denominado *downsampling path* y sus parámetros permanecerán fijos, por lo que utilizaremos la MobileNetV2 pre-entrenada como extractor de características.

Con respecto al *upsampling path*, definiremos una estructura basada convoluciones transpuestas y en un bloque convolucional que definiremos previamente.

*   [Capa convolución transpuesta](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2DTranspose): `Conv2DTranspose(n_filters, kernel_size)` crea una capa con `n_filters` de tamaño `kernel_size` que se aplican a los datos de entrada para producir un tensor de salidas. Si `use_bias` es `True`, se crea un vector de sesgo y se suma a las salidas. Si `activation` no es `None`, también se aplica la función de activación especificada a las salidas. Otros parámetros relevantes:
> * `strides`: un entero o tupla/lista de un entero, especificando el paso de la convolución transpuesta. `strides>1` es incompatible con `dilation_rate>1`.
> * `padding`: `valid`, que significa sin relleno; o `same`, que da como resultado un relleno de ceros uniforme (izquieda/derecha y arriba/abajo). Si `padding='same'`y `strides=1`, la salida tiene el mismo tamaño que la entrada.
> * `activation`: función de activación (`relu`, `sigmoid`, etc.) Por defecto, `activation=None` (es decir, no se utiliza función de activación).
*   **Bloque convolucional:** Definidio ad-hoc para esta arquitectura, está compuesto de una capa convolucional con la función de activación `ReLU`. Permite aplicar una capa de `BatchNormalization` antes de la función de activación, en función de un valor *Booleano* que recibe como parámetro, por lo que este último paso se aplica en una capa independiente.

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, BatchNormalization, Activation, Dropout, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2

def conv2d_block(input_tensor, n_filters, kernel_size = 3, batchnorm = True):
    x = Conv2D(filters = n_filters, kernel_size = kernel_size, kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

def get_model(n_filters = 16, batchnorm = True, dropout = 0.1):

    inputs = Input(shape=(img_width, img_height, n_channels), name="input_image")

    # ENCODER: MobileNetV2
    encoder = MobileNetV2(input_tensor=inputs, include_top=False)

    BASE_WEIGHT_PATH = ('https://github.com/fchollet/deep-learning-models/releases/download/v0.6/')
    model_name = 'mobilenet_%s_%d_tf_no_top.h5' % ('1_0', 224)
    weight_path = BASE_WEIGHT_PATH + model_name
    weights_path = tf.keras.utils.get_file(model_name, weight_path)
    encoder.load_weights(weights_path, by_name=True, skip_mismatch=True)

    skip_connection_names = ["input_image", "block_1_expand_relu", "block_3_expand_relu", "block_6_expand_relu"]
    encoder_output = encoder.get_layer("block_13_expand_relu").output
    x = encoder_output

    x_skip_1 = encoder.get_layer(skip_connection_names[-1]).output  # 224x224
    x_skip_2 = encoder.get_layer(skip_connection_names[-2]).output  # 112x112
    x_skip_3 = encoder.get_layer(skip_connection_names[-3]).output  # 56x56
    x_skip_4 = encoder.get_layer(skip_connection_names[-4]).output  # 28x28

    # DECODER
    u6 = Conv2DTranspose(n_filters * 13, (3, 3), strides = (2, 2), padding = 'same')(x)
    u6 = concatenate([u6, x_skip_1])
    c6 = conv2d_block(u6, n_filters * 13, kernel_size = 3, batchnorm = batchnorm)
    p6 = Dropout(dropout)(c6)

    u7 = Conv2DTranspose(n_filters * 12, (3, 3), strides = (2, 2), padding = 'same')(c6)
    u7 = concatenate([u7, x_skip_2])
    c7 = conv2d_block(u7, n_filters * 12, kernel_size = 3, batchnorm = batchnorm)
    p7 = Dropout(dropout)(c7)

    u8 = Conv2DTranspose(n_filters * 11, (3, 3), strides = (2, 2), padding = 'same')(p7)
    u8 = concatenate([u8, x_skip_3])
    c8 = conv2d_block(u8, n_filters * 11, kernel_size = 3, batchnorm = batchnorm)
    p7 = Dropout(dropout)(c8)

    u9 = Conv2DTranspose(n_filters * 10, (3, 3), strides = (2, 2), padding = 'same')(c8)
    u9 = concatenate([u9, x_skip_4])
    c9 = conv2d_block(u9, n_filters * 10, kernel_size = 3, batchnorm = batchnorm)
    outputs = Conv2D(n_classes, (1, 1), activation='softmax')(c9)

    model = Model(inputs=inputs, outputs=outputs)
    model.summary()

    return model

### **3. Entrenamiento**

Una vez definida la arquitectura del modelo de segmentación semántica, el siguiente paso consiste en configurarlo para el entrenamiento. Para ello definiremos, en primer lugar, el coeficiente de similitud Dice para utilizarlo como métrica de evaluación y para definir la función de pérdida.

*  **Coeficiente de similitud Dice:** Mide el solapamiento entre dos regiones, la real (máscara) y la que predice el modelo.


In [ ]:
import tensorflow.keras.backend as K

def dice_coef(y_true, y_pred, smooth=1e-7):
    y_true_f = K.flatten(K.one_hot(K.cast(y_true, 'int32'), num_classes=n_classes)[Ellipsis,1:])
    y_pred_f = K.flatten(y_pred[...,1:])
    intersect = K.sum(y_true_f * y_pred_f, axis=-1)
    denom = K.sum(y_true_f + y_pred_f, axis=-1)
    return K.mean((2. * intersect / (denom + smooth)))

def dice_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

A continuación, compilaremos el modelo utilizando el método [`compile()`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#compile), siendo estos algunos de sus parámetros más relevantes:

* `loss`: función de pérdida (`mean_squared_error`, `binary_crossentropy`, `categorical_crossentropy`, etc.). En la web de `TensorFlow` puedes encontrar otras [funciones de pérdida](https://www.tensorflow.org/api_docs/python/tf/keras/losses).
* `metrics`: métricas que se evalúan para los datos de entrenamiento y validación (`accuracy`, etc.). En la web de `TensorFlow` puedes encontrar otras [métricas](https://www.tensorflow.org/api_docs/python/tf/keras/metrics).
* `optimizer`: nombre del optimizador (`Adam`, `RMSProp`, etc.) y tasa de aprendizaje (`learning_rate`). En la web de `TensorFlow` puedes encontrar otros [optimizadores](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers).
* `callbacks`: callabacks que se utilizarán durante el entrenamiento (`ReduceLROnPlateau`, `EarlyStopping`, etc). En la web de `TensorFlow` puedes encontrar otros [callbacks](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks).

In [ ]:
from tensorflow.keras.optimizers import Adam

# Crear el modelo
model = get_model(n_filters = 16, batchnorm = True, dropout = 0.1)

# Congelar los parámetros de las capas del codificador
encoder_layers = model.layers[0:-22]
for layer in encoder_layers:
    layer.trainable = False

# Compilar el modelo con el optimizador Adam
opt = Adam(learning_rate=0.001)
model.compile(loss=dice_loss, optimizer=opt, metrics=[dice_coef, 'accuracy'])

Por último, vamos a entrenar el modelo para buscar los parámetros que hagan mínima la función de pérdida. Para ello utilizaremos el método [`fit()`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit), que necesita que le suministremos los datos de entrenamiento y validación, y el número de *epochs*.

En este caso, además, hemos configurado dos callbacks que se utilizarán durante el entrenamiento:

*  [EarlyStopping](tf.keras.callbacks.EarlyStopping): Detiene el entrenamiento cuando una métrica supervisada ha dejado de mejorar.
*  [ReduceLROnPlateau](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ReduceLROnPlateau): Reduce la tasa de aprendizaje cuando una métrica ha dejado de mejorar.

In [ ]:
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stopping = EarlyStopping(monitor='val_dice_coef', patience=20, mode='max')
reduce_lr = ReduceLROnPlateau(monitor='val_dice_coef', factor=0.2, patience=5, min_lr=0.00001)

model.fit(
    train_dataset,
    validation_data = val_dataset,
    epochs=20,
    steps_per_epoch=np.ceil(len(x_train)/batch_size),
    validation_steps=np.ceil(len(x_val)/batch_size),
    callbacks=[early_stopping, reduce_lr]
)

### **4. Predicción**

Una vez entrenado el modelo, es posible utilizarlo para segmentar semánticamente una imagen.

Para ello, vamos a definir dos funciones que permiten visualizar la máscara segmentada de una imagen original y la predicción del modelo.

In [ ]:
def create_mask(pred_mask):
    pred_mask = tf.argmax(pred_mask, axis=-1)
    pred_mask = tf.expand_dims(pred_mask, axis=-1)
    return pred_mask

def display_sample(display_list):
    plt.figure(figsize=(15, 15))
    title = ['True mask', 'Predicted mask']
    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
        plt.imshow(display_list[i])
    plt.show()

Por último, utilizaremos una imagen del conjunto de validación para mostrar un ejemplo de predicción con el modelo previamente entrenado.

In [ ]:
for image, mask in val_dataset.take(3):
    sample_image, sample_mask = image, mask

img = sample_image[0][tf.newaxis, ...]
prediction = model.predict(img)
pred_mask = create_mask(prediction)
display_sample([sample_mask[0], pred_mask[0]])

### **5. Ejercicios**

A continuación, se propone un ejercicio para su resolución.

**EJERCICIO 1**

Dado el modelo original, ¿qué cambios tendríamos que hacer para sustituir la base convolucional de la MobileNetV2 por una VGG16?

In [ ]:
# To-Do: Solución al ejercicio 1